# Mission to Mars

### Scraping Step 1

In [1]:
# Import the Dependencies
import pandas as pd
import requests
import time
from splinter import Browser
from bs4 import BeautifulSoup
import tweepy
import TweepyCredentials # twitter keys and tokens
import pymongo


### NASA Mars News 

In [2]:
# URL of NASA Mars News to be scraped for latest news and the paragraph Title
url_NASA_News = 'https://mars.nasa.gov/news/'

# Set up splinter
exec_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **exec_path)
browser.visit(url_NASA__News)

html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Latest News Title from NASA Mars News Site
news_title = news_soup.find_all('div', class_='content_title')
print(news_title[0].text)

In [2]:
# Latest News Paragraph Text from NASA Mars News Web Site
news_p = news_soup.find_all('div', class_='article_teaser_body')
print(news_p[0].text)

### Mars Space Images - Featured Image

In [5]:
# URL of JPL Mars Space Image to be scraped for featured image
url_images = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Setting up splinter
exec_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **exec_path)
browser.visit(url_images)

In [6]:
# Browse through the pages
time.sleep(5)

# Find and click the full image button
full_img = browser.find_by_id('full_image')
full_img.click()

In [7]:
time.sleep(5)

# Find the more info button and click that
more_info = browser.find_link_by_partial_text('more info')
more_info.click()

In [8]:
# Using BeautifulSoup create an object and parse with 'html.parser'
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [4]:
# find the relative image url
img_url_rel = img_soup.find('figure', class_='lede').find('img')['src']
img_url_rel

In [5]:
# Use the base url to create an absolute url
JPL_link = 'https://www.jpl.nasa.gov'
featured_image_url = JPL_link + img_url_rel
featured_image_url

### Mars Weather

In [11]:
# Twitter API Keys
consumer_key = TweepyCredentials.consumer_key
consumer_secret = TweepyCredentials.consumer_secret
access_token = TweepyCredentials.access_token
access_token_secret = TweepyCredentials.access_token_secret

In [12]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [13]:
# Target User
target_user = "@MarsWxReport"

In [6]:
# Retrive latest tweet
tweet = api.user_timeline(target_user)
mars_weather = tweet[0]['text']
mars_weather

### Mars Facts

In [7]:
# URL of Mars facts to scrape the table containing facts about the planet
url_Mars_Facts = 'https://space-facts.com/mars/'

df = pd.read_html(url_Mars_Facts)[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

In [8]:
# Convert the dataframe to HTML table string
mars_profile_html = df.to_html()
mars_profile_html

### Mars Hemisphere

In [17]:
# Scape the USGS Astrogeology site to obtain high resolution images of Mars each Hemispheres.
url_USGS = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Setting up splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)
browser.visit(url_USGS)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [18]:
# Create an empty list to hold dictionaries of Hemisphere Title with the image url string
hemisphere_image_urls = []

# Get a list of all of the Hemispheres
links = browser.find_by_css("a.product-item")

# Loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # Find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item")[i].click()
    
    # Find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append Hemisphere object to the list
    hemisphere_image_urls.append(hemisphere)
    
    # Backward navigation
    browser.back()

In [9]:
hemisphere_image_urls

In [20]:
browser.quit()